In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop
import numpy as np
import random, sys

text = """
'신의 퀴즈' 류덕환이 서유정을 의심했다.
29일 방송된 케이블채널 OCN 수목드라마 '신의 퀴즈:리부트' 6회에서는 요양병원을 찾아간 한진우(류덕환 분), 강경희(윤주희)의 모습이 그려졌다.
이날 사건의 모든 방향이 원장 신연화(서유정)가 있는 요양병원을 가리키고 있단 게 밝혀졌다. 강경희는 윤현종 누나에게 "현종 씨랑 지금 보신 분들 공통점 아냐. 같은 자살 사이트 회원이었고, 이 요양병원을 거쳐 간 후 실종됐다. 이 병원의 누군가와 관련되어 있을 확률이 높다"라고 밝혔다.
한진우는 신연화를 찾아가 "윤현종의 MAO-A에 대한 또 다른 견해가 있었냐. 특이점이나 합병증 여부"라며 "윤현종 씨에게 수술 자국이 있더라"라고 물었다. 이에 신연화는 "저희 병원엔 수술방이 없다. 당연히 뇌 수술도 하지 못한다"라고 말했다.
그러자 한진우는 "어떻게 알았냐. 뇌쪽 수술이란 거. 전 말씀 드린 적 없는데. 게다가 MAO-A는 뇌 수술과 직접적인 연관도 없고"라고 의심했다.
"""

# 문자를 하나하나 읽어 들이고 ID 붙이기
chars = sorted(list(set(text)))
print('사용되고 있는 문자의 수:', len(chars))
print('사용되고 있는 문자의 수:',  chars )

char_indices = dict((c, i) for i, c in enumerate(chars)) # 문자 → ID
indices_char = dict((i, c) for i, c in enumerate(chars)) # ID → 문자

# 텍스트를 maxlen개의 문자로 자르고 다음에 오는 문자 등록하기
maxlen = 20
step = 3
sentences = []
next_chars = []


In [ ]:

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('학습할 구문의 수:', len(sentences))
print('텍스트를 ID 벡터로 변환합니다...')

X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

In [ ]:
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

# 모델 구축하기(LSTM)
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [ ]:
# 후보를 배열에서 꺼내기
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# 학습시키고 텍스트 생성하기 반복
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('반복 =', iteration)
    model.fit(X, y, batch_size=128, nb_epoch=1) # 
    # 임의의 시작 텍스트 선택하기
    start_index = random.randint(0, len(text) - maxlen - 1)
    # 다양한 다양성의 문장 생성
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('--- 다양성 = ', diversity)
        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('--- 시드 = "' + sentence + '"')
        sys.stdout.write(generated)

        #텍스트 자동 생성
        for i in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            # 다음에 올 문자를 예측하기
            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            # 출력하기
            generated += next_char
            sentence = sentence[1:] + next_char
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()
